# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [40]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


### Compute

In [9]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [17]:
found = False
key = "heart_failure_data"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:

        datastore = ws.get_default_datastore()

        datastore.upload_files(files=["./heart_failure_clinical_records_dataset.csv"],
                                overwrite=True)

        # Create AML Dataset and register it into Workspace
        example_data = "heart_failure_clinical_records_dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(path=(datastore, "heart_failure_clinical_records_dataset.csv"))       
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key)


df = dataset.to_pandas_dataframe()
df

Uploading an estimated of 1 files
Uploading ./heart_failure_clinical_records_dataset.csv
Uploaded ./heart_failure_clinical_records_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [18]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [20]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             **automl_settings
                            )


In [21]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
auto-ml,AutoML_93431f70-da04-4032-982b-67c9c9eb3449,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [22]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [24]:
best_model, fitted_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.51.0.post2, current version:1.49.0
Package:azureml-core, training version:1.51.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.8, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.12, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.51.0, current version:1.49.0
Package:azureml-defaults, training version:1.51.0, current version:1.49.0
Package:azureml-interpret, training version:1.51.0, current version:1.49.0
Package:azureml-mlflow, training version:1.51.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.51.0, current version:1.49.0
Package:azureml-responsibleai, training version:1.51.0, current version:1.49.0
Package:azureml-telemetry, training version:1.51.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.51.0.post1, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.51.0.po

In [36]:
best_model.get_metrics()

{'matthews_correlation': 0.6687219890976582,
 'weighted_accuracy': 0.8688561369808374,
 'average_precision_score_weighted': 0.9337287145422248,
 'balanced_accuracy': 0.8266973407662765,
 'log_loss': 0.3497742095290052,
 'recall_score_macro': 0.8266973407662765,
 'AUC_macro': 0.919739417273971,
 'average_precision_score_macro': 0.9121956052518335,
 'accuracy': 0.8526436781609196,
 'AUC_micro': 0.9249982824679612,
 'AUC_weighted': 0.919739417273971,
 'f1_score_micro': 0.8526436781609196,
 'precision_score_weighted': 0.8658115690935164,
 'f1_score_macro': 0.824438928682814,
 'precision_score_micro': 0.8526436781609196,
 'f1_score_weighted': 0.8497197992531366,
 'norm_macro_recall': 0.6533946815325533,
 'precision_score_macro': 0.8440758342964225,
 'recall_score_weighted': 0.8526436781609196,
 'average_precision_score_micro': 0.9286466077942492,
 'recall_score_micro': 0.8526436781609196,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_93431f70-da04-4032-982b-67c9c9eb3449_4

In [38]:
fitted_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('25', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('extratreesclassifier', ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features=None, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_samples_split=0.056842105263157895, min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False))], verbose=False)), ('21', Pipeline(memory=

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# register the best model
model = best_model.register_model(model_name="best_automl_model", model_path="outputs/model.pkl")

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
inference_config = InferenceConfig(entry_script="inference.py",
                                   environment=inference_env)

service_name = "automl-deployed-model"
deploy_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

deployed_service = Model.deploy(workscapce=ws, name=service_name, models=[model],
                                inference_config=inference_config,
                                deploy_config=deploy_config)

deployed_service.wait_for_deployment(show_output=True)

deployed_service.scoring_uri

In [ ]:
# Enable app insights 

deployed_service.update(enable_app_insights=True)

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
